# Quixo

In [16]:
from game import Game
from random_player import RandomPlayer
from minmax_player import MinmaxPlayer


## Random vs Minmax

In [17]:
g = Game()
player2 = MinmaxPlayer(1)
player1 = RandomPlayer()
winner = g.play(player1, player2, True)
g.print()
print(f"Winner: Player {winner}")

⬜️ ⬜️ ⬜️ ⬜️ ❌ 
⬜️ ⬜️ ⬜️ ⬜️ ⬜️ 
⬜️ ⬜️ ⬜️ ⬜️ ⬜️ 
⬜️ ⬜️ ⬜️ ⬜️ ⬜️ 
⬜️ ⬜️ ⬜️ ⬜️ ⬜️ 
Player 0 moved


TypeError: MinmaxPlayer.__count_consecutive() takes 3 positional arguments but 4 were given

## Minmax vs Random


In [ ]:
g = Game()
player1 = RandomPlayer()
player2 = MinmaxPlayer()
winner = g.play(player1, player2, watch=True)
g.print()
print(f"Winner: Player {winner}")

In [ ]:
# array of all the possible moves for each position

from game import Move
MOVES = {
    #corner moves
    (0, 0): [Move.BOTTOM, Move.RIGHT],
    (0, 4): [Move.BOTTOM, Move.LEFT],
    (4, 0): [Move.TOP, Move.RIGHT],
    (4, 4): [Move.TOP, Move.LEFT],
    
    # high border moves
    (0, 1): [Move.BOTTOM, Move.RIGHT, Move.LEFT],
    (0, 2): [Move.BOTTOM, Move.RIGHT, Move.LEFT],
    (0, 3): [Move.BOTTOM, Move.RIGHT, Move.LEFT],
    
    # low border moves
    (4, 1): [Move.TOP, Move.RIGHT, Move.LEFT],
    (4, 2): [Move.TOP, Move.RIGHT, Move.LEFT],
    (4, 3): [Move.TOP, Move.RIGHT, Move.LEFT],
    
    # left border moves
    (1, 0): [Move.BOTTOM, Move.TOP, Move.RIGHT],
    (2, 0): [Move.BOTTOM, Move.TOP, Move.RIGHT],
    (3, 0): [Move.BOTTOM, Move.TOP, Move.RIGHT],
    
    # right border moves
    (1, 4): [Move.BOTTOM, Move.TOP, Move.LEFT],
    (2, 4): [Move.BOTTOM, Move.TOP, Move.LEFT],
    (3, 4): [Move.BOTTOM, Move.TOP, Move.LEFT],
}

for a, b in MOVES.items():
    print(a, b)

In [42]:
# function to count the number of consecutive pieces in the board

import numpy as np
from scipy.signal import convolve2d
import time

def count_consecutive(matrix, value, length):
    # Create a filter for convolution
    filter_horizontal = np.ones((1, length))
    filter_vertical = np.ones((length, 1))
    filter_diagonal1 = np.eye(length)
    filter_diagonal2 = np.fliplr(filter_diagonal1)
    
    # Create a binary matrix for the given value
    binary_matrix = (matrix == value).astype(int)

    # Convolve the binary matrix with the filters
    horizontal = convolve2d(binary_matrix, filter_horizontal, mode='valid')
    vertical = convolve2d(binary_matrix, filter_vertical, mode='valid')
    diagonal1 = convolve2d(binary_matrix * np.eye(binary_matrix.shape[0], dtype=int), filter_diagonal1, mode='valid')
    diagonal2 = convolve2d(binary_matrix * np.fliplr(np.eye(binary_matrix.shape[0], dtype=int)), filter_diagonal2, mode='valid')

    # # Count the number of times the length appears in the convolutions
    count = 0
    for conv in [horizontal, vertical, diagonal1, diagonal2]:
        count += np.count_nonzero(conv == length)
    
    return count

# Calculate max points for this state (which is an impossible case limit)
matrix = np.array([[1, 1, 1, 1, 1],
                   [1, 1, 1, 1, 1],
                   [1, 1, 1, 1, 1],
                   [1, 1, 1, 1, 1],
                   [1, 1, 1, 1, 1]])

count5 = count_consecutive(matrix, 1, 5)
print("5 consec", count5)
count4 = count_consecutive(matrix, 1, 4) #- count5
print("4 consec", count4)
count3 = count_consecutive(matrix, 1, 3) #- count4 - count5
print('3 consec', count3)
count2 = count_consecutive(matrix, 1, 2) #- count3 - count4
print('2 consec', count2)

5 consec 12
4 consec 12
3 consec 12
2 consec 24


In [30]:
# experiments with matrix rotation with string notation

import numpy as np
def main():
    matrix = np.array([[1, 1, 1, 1, 1],
                    [2, 2, 2, 2, 2],
                    [3, 3, 3, 3, 3],
                    [4, 4, 4, 4, 4],
                    [5, 5, 5, 5, 5]])
    mat_string = ''.join([''.join([str(x) for x in row]) for row in matrix])
    print(mat_string)

    
    mat_rot = [mat_string[i+5*j] for i in range(5) for j in range(5)]
    
    
    print(''.join(mat_rot))
    
    # matrix = np.rot90(matrix, 1)
    # mat_string = ''.join([''.join([str(x) for x in row]) for row in matrix])
    # print(mat_string)
    # matrix = np.rot90(matrix, 1)
    # mat_string = ''.join([''.join([str(x) for x in row]) for row in matrix])
    # print(mat_string)
    # matrix = np.rot90(matrix, 1)
    # mat_string = ''.join([''.join([str(x) for x in row]) for row in matrix])
    # print(mat_string)
    
main()

1111122222333334444455555
1234512345123451234512345


In [33]:
# alternative notation for the states to use strings instead of arrays

matrix = np.array([[1, 1, 1, 1, 1],
                   [1, 1, -1, 1, 1],
                   [1, 1, 1, 1, 1],
                   [1, -1, 1, 1, 1],
                   [1, 1, 1, -1, 1]])

board_str = ''.join([str(cell if cell != -1 else ' ') for cell in matrix.flatten()])
print(board_str)

1111111 11111111 111111 1


In [ ]:
# parameter calculator

from mm_utils import *

evaluation = 25 + 24 * E_PLAYER_COUNT_4 + 12 * E_PLAYER_COUNT_3 
print(evaluation) # result: 325